<h2><font color="#004D7F" size=5>Módulo 2: Bootstrap Aggregation</font></h2>



<h1><font color="#004D7F" size=6> 1. Programación Bootstrap Aggegration </font></h1>

<br><br>
<div style="text-align: right">
<font color="#004D7F" size=3>Manuel Castillo-Cara</font><br>
<font color="#004D7F" size=3>Aprendizaje Automático II</font><br>
<font color="#004D7F" size=3>Universidad Nacional de Educación a Distancia</font>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>


* [1. Introducción](#section1)
   * [1.1. Algoritmo Bootstrap Aggregation](#section11)
   * [1.2. Dataset](#section12)
   * [1.3. Tutorial](#section13)
* [2. Remuestreo de Bootstrap](#section2)
* [3. Caso de estudio: dataset Sonar](#section3)
   * [3.1. Resultados](#section31)
   * [3.2. Comentarios finales](#section32)
* [Ejercicios](#sectionEj)

---

<a id="section0"></a>
# <font color="#004D7F">0. Contexto</font>

Los árboles de decisión son una técnica de modelado predictivo simple y poderosa, pero adolecen de una gran varianza. Esto significa que los árboles pueden obtener resultados muy diferentes con diferentes datos de entrenamiento. Una técnica para hacer que los árboles de decisión sean más robustos y lograr un mejor rendimiento se llama agregación de arranque (Bootstrap aggregation) para abreviar. En este tutorial descubrirás cómo implementar el procedimiento bootstrap con árboles de decisión desde cero con Python. Después de completar este
tutorial, sabrá:
- Cómo crear una muestra bootstrap de su conjunto de datos.
- Cómo hacer predicciones con modelos bootstrap.
- Cómo aplicar bagging a sus propios problemas de modelado predictivo.

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section1"></a>
# <font color="#004D7F"> 1. Introducción</font>

Esta sección proporciona una breve descripción de Bootstrap Aggregation y el conjunto de datos de Sonar que se utilizarán en este tutorial.

<a id="section11"></a>
## <font color="#004D7F"> 1.1. Algoritmo Bootstrap Aggregation</font>

Bootstrap es una muestra de un conjunto de datos con reemplazo. 
- Esto significa que se crea un nuevo conjunto de datos a partir de una muestra aleatoria de un conjunto de datos existente donde se puede seleccionar una fila determinada y agregarla más de una vez a la muestra. 
- Es un enfoque útil para estimar valores como la media de un conjunto de datos más amplio, cuando solo se dispone de un conjunto de datos limitado. 
- Al crear muestras de su conjunto de datos y estimar la media de esas muestras, puede tomar el promedio de esas estimaciones y tener una mejor idea de la verdadera media del problema subyacente.

Este mismo enfoque se puede utilizar con algoritmos de aprendizaje automático que tienen una alta varianza, como los árboles de decisión (CART) presentados anteriormente. Se entrena un modelo separado en cada muestra de datos Bootstrap y el resultado promedio de esos modelos utilizados para hacer predicciones. Esta técnica se llama agregación bootstrap. 
- La varianza significa que el rendimiento de un algoritmo es sensible a los datos de entrenamiento; 
- una varianza alta sugiere que cuanto más se cambian los datos de entrenamiento, más variará el rendimiento del algoritmo.

El rendimiento de los algoritmos de aprendizaje automático de alta varianza, como los árboles de decisión no
podados, se puede mejorar entrenando muchos árboles y tomando el promedio de sus predicciones. Los resultados
suelen ser mejores que los de un único árbol de decisión. Otro beneficio de Bootstrap, además del rendimiento
mejorado, es que los árboles de decisión bagging no pueden adaptarse demasiado al problema. Se pueden
seguir añadiendo árboles hasta que se alcance el máximo rendimiento.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: En Scikit-learn podemos ver los diferentes parámetros que tiene [BaggingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) y [BaggingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html)
</div>

<a id="section12"></a>
## <font color="#004D7F"> 1.2. Dataset</font>

En este tutorial usaremos el conjunto de datos Sonar. 
- Este conjunto de datos implica la discriminación entre minas y rocas. 
- El desempeño de referencia sobre el problema es aproximadamente del 53%. 

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre el dataset [Sonar](https://archive.ics.uci.edu/dataset/151/connectionist+bench+sonar+mines+vs+rocks)
</div>

<a id="section13"></a>
## <font color="#004D7F"> 1.3. Tutorial</font>

Este tutorial se divide en 2 partes:
- Remuestreo de Bootstrap.
- Caso de estudio Sonar.
Estos pasos proporcionan la base que necesita para implementar y aplicar bootstrap aggregation con árboles de decisión a sus propios problemas de modelado predictivo.

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section2"></a> 
# <font color="#004D7F"> 2. Remuestreo de Bootstrap</font>

Comencemos por tener una idea clara de cómo funciona el método bootstrap. 
- Podemos crear una nueva muestra de un conjunto de datos seleccionando aleatoriamente filas del conjunto de datos y agregándolas a una nueva lista. 
- Podemos repetir esto para un número fijo de filas o hasta que el tamaño del nuevo conjunto de datos coincida con una proporción del tamaño del conjunto de datos original.
- Podemos permitir el muestreo con reemplazo al no eliminar la fila que se seleccionó para que esté disponible para selecciones futuras. 

Veamos algunos detalles del código:
- A continuación se muestra una función denominada `subsample()` que implementa este procedimiento. 
- La función `randrange()` del módulo `random` se utiliza para seleccionar un índice de fila aleatorio para agregar a la muestra cada iteración del bucle. 
- El tamaño predeterminado de la muestra es el tamaño del conjunto de datos original.

In [1]:
# Example of subsampling a dataste
from random import seed
from random import randrange

# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio=1.0):
	sample = list()
	n_sample = round(len(dataset) * ratio)
	while len(sample) < n_sample:
		index = randrange(len(dataset))
		sample.append(dataset[index])
	return sample

Podemos utilizar esta función para estimar la media de un conjunto de datos artificial. 
1. Primero, podemos crear un conjunto de datos con 20 filas y una sola columna de números aleatorios entre 0 y 9 y calcular el valor medio. 
2. Luego podemos hacer muestras bootstrap del conjunto de datos original, calcular la media y repetir este proceso hasta que tengamos una lista de medias. 
3. Tomar el promedio de estas medias muestrales debería darnos una estimación sólida de la media de todo el conjunto de datos.

Veamos el código:
- Cada muestra de arranque se crea como una muestra del 10% del conjunto de datos de 20 observaciones original (o 2 observaciones). 
- Luego experimentamos creando 1, 10, 100 muestras bootstrap del conjunto de datos original, calculamos su valor medio y luego promediamos todos esos valores medios estimados.

In [2]:
# Calculate the mean of a list of numbers
def mean(numbers):
	return sum(numbers) / float(len(numbers))

# Test subsampling a dataset
seed(1)
# True mean
dataset = [[randrange(10)] for i in range(20)]
print('True Mean: %.3f' % mean([row[0] for row in dataset]))
# Estimated means
ratio = 0.10
for size in [1, 10, 100]:
	sample_means = list()
	for i in range(size):
		sample = subsample(dataset, ratio)
		sample_mean = mean([row[0] for row in sample])
		sample_means.append(sample_mean)
	print('Samples=%d, Estimated Mean: %.3f' % (size, mean(sample_means)))

True Mean: 4.500
Samples=1, Estimated Mean: 4.000
Samples=10, Estimated Mean: 4.700
Samples=100, Estimated Mean: 4.570


Al ejecutar el ejemplo se imprime el valor medio original que pretendemos estimar. Luego podemos ver la media
estimada a partir de diferentes números de muestras de arranque. Podemos ver que con 100 muestras logramos una
buena estimación de la media

En lugar de calcular el valor medio, podemos crear un modelo a partir de cada submuestra. Veamos cómo podemos combinar las predicciones de múltiples modelos bootstrap.

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section3"></a> 
# <font color="#004D7F"> 3. Caso de estudio: dataset Sonar</font>

En esta sección, aplicaremos el algoritmo Bagging al conjunto de datos de Sonar. 
1. Primero se carga el conjunto de datos, los valores de cadena se convierten a numéricos y la columna de salida se convierte de cadenas a valores enteros de 0 a 1. 
    - Esto se logra con las funciones auxiliares `load_csv()`, `str_column_to_float()` y `str_columna_to_int()` para cargar y preparar el conjunto de datos.
2. Usaremos _k_-fold para estimar el rendimiento del modelo aprendido en datos invisibles. 
    - Esto significa que construiremos y evaluaremos _k_ modelos y estimaremos el rendimiento como el error medio del modelo. 
    - El Accuracy se utilizará para evaluar cada modelo. 
    - Estos comportamientos se proporcionan en las funciones `cross_validation_split()`, `accuracy_metric()`
y `evaluate_algorithm()`.
3. También usaremos una implementación del algoritmo CART adaptado para empaquetar con las funciones auxiliares del Capítulo 11 , incluyendo:
    - `test split()` para dividir un conjunto de datos en grupos, 
    - `gini_index()` para evaluar un punto de división, 
    - `get_split()` para encontrar un punto de división óptimo, 
    - `to_terminal()`, `split()` y `build_tree()` se usan para crear un único árbol de decisión, 
    - `predict()` para hacer una predicción con un árbol de decisión y
    - `subsample()` descrita en el paso anterior para hacer una submuestra del conjunto de datos de entrenamiento.
4. Se desarrolla una nueva función denominada `bagging_predict()` que se encarga de realizar una predicción con cada árbol de decisión y combinar las predicciones en un único valor de retorno. 
    - Esto se logra seleccionando la predicción más común de la lista de predicciones realizadas por los árboles bagging.
5. Finalmente, se desarrolla una nueva función llamada `bagging()` que es responsable de crear las muestras del conjunto de datos de entrenamiento, entrenar un árbol de decisión en cada una y luego hacer predicciones en el conjunto de test utilizando la lista de árboles bagging. 

El ejemplo completo se enumera a continuación.

In [4]:
# Bagging Algorithm on the Sonar dataset
from random import seed
from random import randrange
from csv import reader

# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for _ in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

# Select the best split point for a dataset
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	for index in range(len(dataset[0])-1):
		for row in dataset:
		# for i in range(len(dataset)):
		# 	row = dataset[randrange(len(dataset))]
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

# Create a random subsample from the dataset with replacement
#def subsample(dataset, ratio):
#	sample = list()
#	n_sample = round(len(dataset) * ratio)
#	while len(sample) < n_sample:
#		index = randrange(len(dataset))
#		sample.append(dataset[index])
#	return sample

# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
	predictions = [predict(tree, row) for tree in trees]
	return max(set(predictions), key=predictions.count)

# Bootstrap Aggregation Algorithm
def bagging(train, test, max_depth, min_size, sample_size, n_trees):
	trees = list()
	for _ in range(n_trees):
		sample = subsample(train, sample_size)
		tree = build_tree(sample, max_depth, min_size)
		trees.append(tree)
	predictions = [bagging_predict(trees, row) for row in test]
	return(predictions)

# Test bagging on the sonar dataset
seed(1)
# load and prepare data
filename = 'data/sonar.all-data.csv'
dataset = load_csv(filename)
# convert string attributes to integers
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
max_depth = 6
min_size = 2
sample_size = 0.50
for n_trees in [1, 5, 10, 50]:
	scores = evaluate_algorithm(dataset, bagging, n_folds, max_depth, min_size, sample_size, n_trees)
	print('Trees: %d' % n_trees)
	print('Scores: %s' % scores)
	print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Trees: 1
Scores: [87.8048780487805, 65.85365853658537, 65.85365853658537, 65.85365853658537, 73.17073170731707]
Mean Accuracy: 71.707%
Trees: 5
Scores: [60.97560975609756, 80.48780487804879, 78.04878048780488, 82.92682926829268, 63.41463414634146]
Mean Accuracy: 73.171%
Trees: 10
Scores: [60.97560975609756, 73.17073170731707, 82.92682926829268, 80.48780487804879, 68.29268292682927]
Mean Accuracy: 73.171%
Trees: 50
Scores: [63.41463414634146, 75.60975609756098, 80.48780487804879, 75.60975609756098, 85.36585365853658]
Mean Accuracy: 76.098%


<a id="section31"></a> 
## <font color="#004D7F"> 3.1. Resultados</font>

- Se utilizó un valor _k_ de 5 para la validación cruzada, dando a cada pliegue $\frac{208}{5} = 41.6$ o justo 40 registros que se evaluarán en cada iteración.
- Se construyeron árboles con una profundidad máxima de 6 y un número mínimo de filas de entrenamiento en cada nodo de 2. 
- Se crearon muestras del conjunto de datos de entrenamiento con un 50% del tamaño del conjunto de datos original. Esto fue para forzar cierta variedad en las submuestras del conjunto de datos utilizadas para entrenar cada árbol. 
- El valor predeterminado para el bagging es que el tamaño de los conjuntos de datos de muestra coincida con el tamaño del conjunto de datos de entrenamiento original.
- Se evaluó una serie de 4 números diferentes de árboles para mostrar el comportamiento del algoritmo.
- Se imprimen el accuracy de cada fold y accuracy medio de cada configuración. Podemos ver una tendencia de un ligero aumento en el rendimiento a medida que aumenta el número de árboles.

<a id="section32"></a> 
## <font color="#004D7F"> 3.2. Comentarios finales</font>

Una dificultad de este método es que aunque se construyen árboles profundos, los árboles bagging que se crean son muy similares. A su vez, las predicciones realizadas por estos árboles también son similares y disminuye la alta varianza que deseamos entre los árboles entrenados en diferentes muestras del conjunto de datos de entrenamiento .

Esto se debe al algoritmo greedy utilizado en la construcción de los árboles seleccionando puntos de división iguales o similares. El tutorial intentó reinyectar esta variación restringiendo el tamaño de la muestra utilizada para entrenar cada árbol. Una técnica más sólida consiste en restringir las características que pueden evaluarse al crear cada punto de división. Este es el método utilizado en el algoritmo Random Forest.

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="sectionEj"></a>
<h3><font color="#004D7F" size=6> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Ejercicios</font></h3>

Se proponen las siguientes actividades para consolidar el aprendizaje.

# <font color="#004D7F" size=5>Ejercicio 1</font>
__Ajustar el ejemplo__. Explore diferentes configuraciones para la cantidad de árboles e incluso configuraciones de árbol individuales para ver si puede mejorar aún más los resultados.

# <font color="#004D7F" size=5>Ejercicio 2</font>
__Bag de otro algoritmo__. Se pueden utilizar otros algoritmos con bagging. Por ejemplo, un algoritmo de _k_-nearest neighbor con un valor bajo de _k_ tendrá una varianza alta y es un buen candidato para el bagging.

# <font color="#004D7F" size=5>Ejercicio 3</font>
__Problema de Regresión__. Bagging se puede utilizar con árboles de regresión. En lugar de predecir el valor de clase más común del conjunto de predicciones, puede devolver el promedio de las predicciones de los árboles bagging. Experimente con problemas de regresión.

---

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>